# Exploración del dataset


In [1]:
import pandas as pd
import rdflib
import numpy

In [2]:
df = pd.read_csv('dataset/raw.csv.gz', dialect='excel', keep_default_na=False, dtype=str, index_col=0)

El dataset tiene un indice que no se corresponde con la cantidad de filas del dataset, sino con el dataset original. Para acceder a las filas se puede usar `.loc[indice]` en lugar de `.iloc[indice]`.

In [3]:
df.index

Index([     0,      1,      2,      3,      4,      5,      6,      7,      8,
            9,
       ...
       764978, 764981, 764982, 764983, 764985, 764986, 764987, 764988, 764991,
       764992],
      dtype='int64', length=55227)

In [4]:
df.loc[1]  # .loc hace referencia al indice

_validation                                                                          
address                                                                      Calle 30
advertiser_id                                                               209733492
advertiser_name                                                  AMITRANO PROPIEDADES
age                                                                                  
bath_amnt                                                                           4
bed_amnt                                                                            4
bed_ratio                                                                            
covered_ratio                                                       675.1054852320675
covered_surface                                                                   237
covered_surface_unit                                                               m²
currency                                              

In [5]:
df.iloc[2001] # .iloc hace referencia al número de fila dentro del dataset

_validation                                                                          
address                                                                   barcelo 630
advertiser_id                                                                64093834
advertiser_name                                                                      
age                                                                                  
bath_amnt                                                                           1
bed_amnt                                                                             
bed_ratio                                                                            
covered_ratio                                                                   400.0
covered_surface                                                                   550
covered_surface_unit                                                               m²
currency                                              

Obtener mis las lineas en las que tengo que trabajar.

In [6]:
df.iloc[2000:2200]

,_validation,address,advertiser_id,advertiser_name,age,bath_amnt,bed_amnt,bed_ratio,covered_ratio,covered_surface,...,toilet_amnt,total_ratio,total_surface,total_surface_unit,transaction,uncovered_surface,uncovered_surface_unit,url,year_built,dup_candidates
2000,,Barrio Buenos Aires Village 1300,17635184,Tizado,1.0,3,4,nan,nan,230.0,...,,nan,230.0,m²,Venta,,,https://www.zonaprop.com.ar/propiedades/clasif...,2023,2000 343873
2001,,barcelo 630,64093834,,,1,,,400.0,550,...,,400.0,550,m²,Venta,,,https://api.mercadolibre.com/items/MLA1550225184,,2001
2002,,Avenida Mitre 1699 7° A y B,702631104,Nicolás Palacios Propiedades,,3,4,55000.0,2037.037037037037,108,...,,1833.3333333333333,120,m²,Venta,,,https://api.mercadolibre.com/items/MLA1527108448,,2002 611877
2003,,Galicia 359 0,230444916,ABRAHAM PAZ & ASOC,,1,2,110000.0,2750.0,80,...,,2200.0,100,m²,Venta,,,https://api.mercadolibre.com/items/MLA1542109896,,798 2003
2004,,"Calle El Salvador 70, Villa Dominico, Provinci...",148240334,,,2,2,59000.0,1311.111111111111,90,...,,1311.111111111111,90,m²,Venta,,,https://api.mercadolibre.com/items/MLA1401805687,,2004 377359 610865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2195,,España 100,162369076,German Berretti Propiedades Palermo,,1,3,41666.666666666664,1785.7142857142858,70,...,,1644.7368421052631,76,m²,Venta,,,https://api.mercadolibre.com/items/MLA1402274505,,2195 611087
2196,,Flaubert al al 1300,30047089,ZIPCODE,7.0,5,4,122500.0,2085.1063829787236,235.0,...,1,1814.8148148148148,270.0,m²,Venta,,,https://www.zonaprop.com.ar/propiedades/clasif...,2017,2196 345370 631278
2197,,Avenida Presidente Juan Domingo Perón 300,30002788,Laura Palini,24.0,1,2,26000.0,962.9629629629629,54.0,...,,928.5714285714286,56.0,m²,Venta,,,https://www.zonaprop.com.ar/propiedades/clasif...,2000,1983 2197 324695
2198,,Av. Mitre al 5900,173284018,Young Propiedades,20.0,1,2,62500.0,1811.5942028985507,69,...,,1811.5942028985507,69,m²,Venta,,,https://api.mercadolibre.com/items/MLA1401153555,2004,2074 2198 611042 611043 611044 611045 611046 6...


☝️ Chequear que tenga 200 filas

In [14]:
#Obtener los candidatos duplicados para una fila

In [176]:
df.loc[2170].dup_candidates

'2070 2170 648173 648798 651360'

In [168]:
#Chequear las descripciones de una fila

In [177]:
candidates = [int(x) for x in df.iloc[2170].dup_candidates.split()] # Esto devuelve una lista de indices [990, 1200]
df.loc[candidates] # Esto devuelve las filas de un dataset en esa lista de indices

,_validation,address,advertiser_id,advertiser_name,age,bath_amnt,bed_amnt,bed_ratio,covered_ratio,covered_surface,...,toilet_amnt,total_ratio,total_surface,total_surface_unit,transaction,uncovered_surface,uncovered_surface_unit,url,year_built,dup_candidates
2070,,Conesa 900,30397286,RE/MAX Data Work,0,1,1,74000.0,1321.4285714285713,56.0,...,,1275.8620689655172,58.0,m²,Venta,,,https://www.zonaprop.com.ar/propiedades/clasif...,2024,2070 2170 648173 648798 651360
2170,,Conesa 900,30397286,RE/MAX Data Work,0,1,2,42000.0,1333.3333333333333,63.0,...,,1292.3076923076924,65.0,m²,Venta,,,https://www.zonaprop.com.ar/propiedades/clasif...,2024,2070 2170 648173 648798 651360
648173,,,268310,,1.0,1,2,42000.0,1333.3333333333333,63.0,...,,1333.3333333333333,,,Venta,,,https://www.argenprop.com/departamento-en-vent...,2023,2891 324548 324621 627425 648173 648798 650279...
648798,,,268310,,1.0,1,1,74000.0,1321.4285714285713,56.0,...,,1321.4285714285713,,,Venta,,,https://www.argenprop.com/departamento-en-vent...,2023,2891 324548 324621 627425 648173 648798 650279...
651360,,,268310,,1.0,1,,,1250.0,32.0,...,,1250.0,,,Venta,,,https://www.argenprop.com/departamento-en-vent...,2023,2891 324548 324621 627425 648173 648798 650279...


In [178]:
# muestra las descripciones de cada aviso junto con sus indidces...
for idx, desc in zip(candidates,df.loc[candidates]['description']):
    print(f' {idx}   ******************')
    print(desc)
    print()

 2070   ******************
Corredor Responsable: LEONARDO MISURACA CMCPSM 2568 / CUCICBA 7970 - Contacto: Adriana Olivera - MLS ID # En la zona mas codiciada de Muñiz, encontramos este edificio de 12 pisos ,con dtos de 1 ,2 y 3 ambientes ,cuenta con rampa para discapacitados, un lobby muy luminoso ,ascensores de ultima generación y espacios descubiertos y cubiertos para autos. Departamento 2 ambientes con posibilidad por su doble ventilación del balcón y sus ventanas de lograr un segundo dormitorio. Cocina completa con bajo mesada y alacenas. Dormitorio con placards. Amplia sala social. Balcón Opcional Cochera No esperes mas ,vamos a visitarlo!!!. Entrega programada Febrero 2024. Remax Argentina S.R.L. no ejerce el corretaje inmobiliario. El presente sitio web es una plataforma en donde cada oficina inmobiliaria independiente que contrata los servicios RE/MAX puede publicar las propiedades a su cargo. Cada oficina es de propiedad y gestión independiente, por lo que Remax Argentina S.R.

## El curado de los datos
Lo que debemos hacer es revisar los avisos a ver si los candidatos realmente están duplicados o no. -+En caso de no estar duplicados, la columna dup_candidates debe ser corregida. Suponga que estamos revisando la fila 199. El valor de la columna `dup_candidates` para la fila 199 es `'199 305206 624765'`, es decir que las filas 305206 y 624765 son duplicadas de la 199. No es el caso en este ejemplo, pero supongamos que la fila 305206 no hace referencia a la misma propiedad. Entonces, la versión corregida para `dup_candidates` debería contener solo `'199 624765'`. Por el contrario, si efectivamente todas las ocurrencias hablan de la misma propiedad, entonces la versión corregida para `dup_candidates` debería quedar exactamente igual, es decir `'199 305206 624765'`.

La idea es generar una nueva lista de 200 strings, con las columnas revisadas y agregarla a un nuevo dataset.

In [8]:
# Inserte su metodología para detectar duplicados aquí!
# El resultado debe ser una lista de 200 strings.

revised = ['']*200
revised[0] = '2000 343873'
revised[1] = '2001'
revised[2] = '2002 611877'
revised[3] = '2003'
revised[4] = '2004 377359 610865'
revised[5] = '2005'
revised[6] = '2006'
revised[7] = '2007'
revised[8] = '2008 2027 3181 346971 348820 349112 647371 647495 647592'
revised[9] = '2009 348906'
revised[10] = '2010 345638'
revised[11] = '2011 8607 348261 348651 629048 647127'
revised[12] = '2012'
revised[13] = '2013'
revised[14] = '2014 380960 610930'
revised[15] = '2015 374761 610866 610867'
revised[16] = '2016 374494'
revised[17] = '2017 610929'
revised[18] = '2018'
revised[19] = '2019 629123'
revised[20] = '2020 344777'
revised[21] = '2021 324687'
revised[22] = '2022'
revised[23] = '2023'
revised[24] = '2024 630957'
revised[25] = '2025 338032 651033'
revised[26] = '2026 367973'
revised[27] = '2008 2027 3181 346971 348820 349112 647371 647495 647592'
revised[28] = '2028'
revised[29] = '2029 343595 629779'
revised[30] = '2030 629225'
revised[31] = '2031 647053'
revised[32] = '2032 376062'
revised[33] = '2033 373522'
revised[34] = '2034 383477'
revised[35] = '2035 610829'
revised[36] = '1159 2036'
revised[37] = '2037 3715 610843 375583 377536 610416'
revised[38] = '2038 369576'
revised[39] = '2039'
revised[40] = '2040'
revised[41] = '2041 610807'
revised[42] = '2042 610827'
revised[43] = '2043 610828'
revised[44] = '2044'
revised[45] = '2045 2178 374709 610830'
revised[46] = '2046'
revised[47] = '2047 631246'
revised[48] = '1088 2048 2210 2228 2353 2478 2654 2672 2923 3262 3566 346843 346976 347052 348472 348475 348801' 
revised[49] =  '2049 627926 650721'
revised[50] = '2050 610833 610934'
revised[51] = '2051 610935'
revised[52] = '2052'
revised[53] = '2053'
revised[54] = '2054 610808'
revised[55] = '2055 345823'
revised[56] = '2056 345135'
revised[57] = '2057 345564'
revised[58] = '2058 346393 644547'
revised[59] = '2059 344370'
revised[60] = '2060 346769 628193'
revised[61] = '2061'
revised[62] = '2062'
revised[63] = '2063 378213'
revised[64] = '1054 2064 376888 378298 610563 610912 611077'
revised[65] = '2065 343040 629917'
revised[66] = '2066 2262 2932 349547 351720 627300 627909'
revised[67] = '2067 344762'
revised[68] = '1988 2068 2225 3246 344691 347103 347576 347579'
revised[69] = '2069 324959 650571'
revised[70] = '2070 2170 648173 648798 651360'
revised[71] = '2071 345347 631608'
revised[72] = '2072 366966 381445 611073'
revised[73] = '2073 612598'
revised[74] = '2445 2074 2434 2436 2440 2471 2472 2474 2477 2484 611042 611043 611044 611045 611046 611071 611214 611244 611245 611246 611247 611248 611249 611250 611251 612558 612603'
revised[75] = '2075'
revised[76] = '2076'
revised[77] = '2077 4798 372705 379695 609771 611079'
revised[78] = '2078'
revised[79] = '2079 3321 346813 347301 628633 646580'
revised[80] = '2080 612607'
revised[81] = '2081'
revised[82] = '339 620 1364 2082 3433 374671 375628 379424 382425'
revised[83] = '2083'
revised[84] = '2084 610663'
revised[85] = '2085'
revised[86] = '2086'
revised[87] = '2087'
revised[88] = '2088 612599'
revised[89] = '2089'
revised[90] = '2090'
revised[91] = '1006 2091 3552'
revised[92] = '2092 608945'
revised[93] = '2093 611032'
revised[94] = '2094 611026'
revised[95] = '2095 630448'
revised[96] = '2096 348742 647744'
revised[97] = '2097 345749 631292'
revised[98] = '2098'
revised[99] = '2099 651727'
revised[100] = '2100 349073 647669'
revised[101] = '2101 344711 630683'
revised[102] = '2102 343871 631183'
revised[103] = '2103'
revised[104] = '2104'
revised[105] = '2105'
revised[106] = '2106'
revised[107] = '2107'
revised[108] = '2108 2143'
revised[109] = '2109'
revised[110] = '2110 383827 612600'
revised[111] = '2111'
revised[112] = '2112'
revised[113] = '2113 337451 651263'
revised[114] = '2114 626993 650993'
revised[115] = '2115 3504 341826 346431 644598'
revised[116] = '2116 341871'
revised[117] = '2117'
revised[118] = '2118 381584'
revised[119] = '2119'
revised[120] = '2120 611852'
revised[121] = '2121 377133'
revised[122] = '2122 612412'
revised[123] = '2123 373564 611872'
revised[124] = '2124 611856'
revised[125] = '2125 611864'
revised[126] = '2126 650363'
revised[127] = '2127 343042 629946'
revised[128] = '2128'
revised[129] = '2129 346747 647914'
revised[130] = '2130'
revised[131] = '2131 647378'
revised[132] = '2132 348001 646906'
revised[133] = '2133 382677'
revised[134] = '2134'
revised[135] = '2135'
revised[136] = '2136 385074'
revised[137] = '2137 372641'
revised[138] = '2138 611857'
revised[139] = '2139 385035'
revised[140] = '2140'
revised[141] = '2141'
revised[142] = '2142 384471 611853'
revised[143] = '2108 2143'
revised[144] = '2144 611865'
revised[145] = '2145 2156 2159 377056'
revised[146] = '2146 382821 611880'
revised[147] = '2147 611879'
revised[148] = '2148 343458 645658'
revised[149] = '2149 350134 627657'
revised[150] = '2150 349119 647493'
revised[151] = '2151 346946 647354'
revised[152] = '2152 349701 649268'
revised[153] = '2153 3225 347921 348767'
revised[154] = '2154 374454'
revised[155] = '2155 611858'
revised[156] = '2159 2145 2156 377056'
revised[157] = '2157 346315 630507'
revised[158] = '2158 348238 629047'
revised[159] = '2159 2145 2156 377056'
revised[160] = '2160 610848'
revised[161] = '2161 373141 610933'
revised[162] = '2162 611841'
revised[163] = '2163 384871 610834'
revised[164] = '2164'
revised[165] = '2165 610849'
revised[166] = '2166'
revised[167] = '2167 346605'
revised[168] = '2168 626886'
revised[169] = '2169 650225' 
revised[170] = '2170 648173'
revised[171] = '2171 349405 648279'
revised[172] = '2172 369763 611837'
revised[173] = '2173'
revised[174] = '2174 367455 610824'
revised[175] = '2175'
revised[176] = '2176 610850 611640 612656'
revised[177] = '2177'
revised[178] = '2045 2178 374709 610830'
revised[179] = '2179'
revised[180] = '2180'
revised[181] = '2181 381446'
revised[182] = '2182 610851 611338 611339 611340 611607 611608 611630 612529 612531'
revised[183] = '2183 344565 631722'
revised[184] = '2184 646584'
revised[185] = '2185 381359'
revised[186] = '2186 371826 610810'
revised[187] = '2187'
revised[188] = '2188 2275 348105 629121 629323'
revised[189] = '2189'
revised[190] = '2190 337340 651135'
revised[191] = '2191 2507 2958 348701 628682 628683 647727'
revised[192] = '2192 2893 3277 627097 627250 627299 646810'
revised[193] = '2193 379333 610844'
revised[194] = '2194 611086'
revised[195] = '2195 611087'
revised[196] = '2196 345370 631278'
revised[197] = '1983 2197 324695'
revised[198] = '2074 2198 611042 611043 611044 611045 611046 611071 612603'
revised[199] = '2199'


La nueva lista de strings puede tener valores en blanco (un string vacio `''`). Esto indica que esa fila no fue revisada. Además, el orden de los indices no importa, por lo tanto `'199 305206 624765'` es igual a `'199 624765 305206'` y a `'305206 624765 199'`. 

## Forma del entregable
El entregable será un nuevo dataset con una nueva columna llamada `dup_curated` el cual contrendrá un string nuevo por cada fila. Este string representará las filas que están duplicadas que han sido verificadas. Si las filas con indice 199, 305206 y 624765 están efectivamente duplicadas, como en el ejemplo anterior, la nueva columna debe quedar igual, es decir, para la fila con indice 199 tendrá el valor

`199 305206 624765`

Una vez que estemos contentos con la nueva lista revisada, vamos a crear un nuevo dataset y a agregar la nueva columna. Si el dataset no tiene la misma longitud que la lista, nos va a dar error! Ojo con eso y ojo con el orden de los elementos de la lista. 

In [9]:
newdf = df.iloc[2000:2200][['dup_candidates']]
newdf

,dup_candidates
2000,2000 343873
2001,2001
2002,2002 611877
2003,798 2003
2004,2004 377359 610865
...,...
2195,2195 611087
2196,2196 345370 631278
2197,1983 2197 324695
2198,2074 2198 611042 611043 611044 611045 611046 6...


In [10]:
newdf['dup_curated'] = revised

In [11]:
newdf

,dup_candidates,dup_curated
2000,2000 343873,2000 343873
2001,2001,2001
2002,2002 611877,2002 611877
2003,798 2003,2003
2004,2004 377359 610865,2004 377359 610865
...,...,...
2195,2195 611087,2195 611087
2196,2196 345370 631278,2196 345370 631278
2197,1983 2197 324695,1983 2197 324695
2198,2074 2198 611042 611043 611044 611045 611046 6...,2074 2198 611042 611043 611044 611045 611046 6...


In [12]:
grupo_nro = 11
newdf.to_csv(f'dataset/entrega_tis_grupo_{grupo_nro:11}.csv.gz', escapechar='"')